# 2024 날씨 빅데이터 콘테스트 

## 전력 분야 - 기상에 따른 공동주택 전력수요 예측 개선 

In [33]:
import pandas as pd
import os 

import matplotlib.pyplot as plt

In [34]:
df = pd.read_csv('./data/electric_train_cp949.csv', encoding='cp949', index_col=0)
df['electric_train.tm'] = pd.to_datetime(df['electric_train.tm'])

print(df.shape)

(7593355, 16)


In [35]:
elec_cols = ['electric_train.'+ a for a in ['tm', 'hh24', 'weekday', 'week_name', 'sum_qctr', 'n', 'sum_load', 'n_mean_load', 'elec']]
df_elec = df[elec_cols]
df_elec.head()

,electric_train.tm,electric_train.hh24,electric_train.weekday,electric_train.week_name,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec
1,2021-01-01 01:00:00,1,4,0,6950,11,751.32,68.606449,99.56
2,2021-01-01 02:00:00,2,4,0,6950,11,692.60,68.606449,91.78
3,2021-01-01 03:00:00,3,4,0,6950,11,597.48,68.606449,79.17
4,2021-01-01 04:00:00,4,4,0,6950,11,553.48,68.606449,73.34
5,2021-01-01 05:00:00,5,4,0,6950,11,526.24,68.606449,69.73


In [36]:
weat_cols = ['electric_train.'+ a for a in ['num', 'stn', 'nph_ta', 'nph_hm', 'nph_ws_10m', 'nph_rn_60m', 'nph_ta_chi']]
df_weat = df[weat_cols]
df_weat.head()

,electric_train.num,electric_train.stn,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi
1,4821,884,2.2,62.7,1.8,0.0,-1.0
2,4821,884,2.3,63.1,2.1,0.0,-0.6
3,4821,884,2.2,62.4,2.5,0.0,-1.3
4,4821,884,1.7,63.5,1.7,0.0,-0.2
5,4821,884,1.7,63.0,1.6,0.0,-0.8


In [37]:
reset_order_cols = elec_cols + weat_cols

df_new = df[reset_order_cols]

df_new.head()

,electric_train.tm,electric_train.hh24,electric_train.weekday,electric_train.week_name,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec,electric_train.num,electric_train.stn,electric_train.nph_ta,electric_train.nph_hm,electric_train.nph_ws_10m,electric_train.nph_rn_60m,electric_train.nph_ta_chi
1,2021-01-01 01:00:00,1,4,0,6950,11,751.32,68.606449,99.56,4821,884,2.2,62.7,1.8,0.0,-1.0
2,2021-01-01 02:00:00,2,4,0,6950,11,692.60,68.606449,91.78,4821,884,2.3,63.1,2.1,0.0,-0.6
3,2021-01-01 03:00:00,3,4,0,6950,11,597.48,68.606449,79.17,4821,884,2.2,62.4,2.5,0.0,-1.3
4,2021-01-01 04:00:00,4,4,0,6950,11,553.48,68.606449,73.34,4821,884,1.7,63.5,1.7,0.0,-0.2
5,2021-01-01 05:00:00,5,4,0,6950,11,526.24,68.606449,69.73,4821,884,1.7,63.0,1.6,0.0,-0.8


In [57]:
df_train = df[df['electric_train.tm'] < '2023-01-01']

targets = ['electric_train.sum_qctr', 'electric_train.n', 'electric_train.sum_load', 'electric_train.n_mean_load']

df_val = df[df['electric_train.tm'] >= '2023-01-01'].drop(targets, axis=1)
# df_val는 다른 features에서 계산을 통해 구한 후 채워넣야 할듯? <-- 훈련 데이터에서는 사용하기 때문 (내 생각.....)

In [58]:
target = 'electric_train.elec'

In [40]:
targets.append(target)
targets = ['electric_train.tm'] + targets


In [41]:
df_targets = df_train[targets]

df_targets

,electric_train.tm,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec
1,2021-01-01 01:00:00,6950,11,751.32,68.606449,99.56
2,2021-01-01 02:00:00,6950,11,692.60,68.606449,91.78
3,2021-01-01 03:00:00,6950,11,597.48,68.606449,79.17
4,2021-01-01 04:00:00,6950,11,553.48,68.606449,73.34
5,2021-01-01 05:00:00,6950,11,526.24,68.606449,69.73
...,...,...,...,...,...,...
7593350,2022-12-31 19:00:00,34200,23,6851.72,225.461986,132.13
7593351,2022-12-31 20:00:00,34200,23,6779.84,225.461986,130.74
7593352,2022-12-31 21:00:00,34200,23,6802.40,225.461986,131.18
7593353,2022-12-31 22:00:00,34200,23,6706.68,225.461986,129.33


In [50]:
df_targets_2020 = df_targets[df_targets['electric_train.tm'].dt.year == 2020]
df_targets_2021 = df_targets[df_targets['electric_train.tm'].dt.year == 2021]
df_targets_2022 = df_targets[df_targets['electric_train.tm'].dt.year == 2022]

In [51]:
df_targets_2020.head()

,electric_train.tm,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec
8761,2020-01-01 01:00:00,42250,58,4468.72,79.776051,96.58
8762,2020-01-01 02:00:00,42250,58,4101.00,79.776051,88.63
8763,2020-01-01 03:00:00,42250,58,3724.60,79.776051,80.50
8764,2020-01-01 04:00:00,42250,58,3484.40,79.776051,75.31
8765,2020-01-01 05:00:00,42250,58,3409.12,79.776051,73.68


In [44]:
# df_targets_2020.reset_index(drop=True, inplace=True)

# df_targets_2020.head()

,electric_train.tm,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec
0,2020-01-01 01:00:00,42250,58,4468.72,79.776051,96.58
1,2020-01-01 02:00:00,42250,58,4101.00,79.776051,88.63
2,2020-01-01 03:00:00,42250,58,3724.60,79.776051,80.50
3,2020-01-01 04:00:00,42250,58,3484.40,79.776051,75.31
4,2020-01-01 05:00:00,42250,58,3409.12,79.776051,73.68


In [52]:
df_targets_2020_1am = df_targets_2020[df_targets_2020['electric_train.tm'].dt.hour == 1]

df_targets_2020_1am.head()

,electric_train.tm,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec
8761,2020-01-01 01:00:00,42250,58,4468.72,79.776051,96.58
8785,2020-01-02 01:00:00,42250,58,4358.52,79.776051,94.20
8809,2020-01-03 01:00:00,42250,58,4524.72,79.776051,97.79
8833,2020-01-04 01:00:00,42250,58,4518.44,79.776051,97.65
8857,2020-01-05 01:00:00,42250,58,4490.52,79.776051,97.05


In [53]:
df_targets_2020_1am.tail()

,electric_train.tm,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec
7575716,2020-12-27 01:00:00,31450,25,4818.40,183.102334,105.26
7575740,2020-12-28 01:00:00,31450,25,4546.88,183.102334,99.33
7575764,2020-12-29 01:00:00,31450,25,4587.52,183.102334,100.22
7575788,2020-12-30 01:00:00,31450,25,4707.20,183.102334,102.83
7575812,2020-12-31 01:00:00,31450,25,4973.16,183.102334,108.64


In [54]:
df_targets_2020_1am_day = df_targets_2020_1am[df_targets_2020_1am['electric_train.tm'].between('2020-01-01','2020-01-02')]
df_targets_2020_1am_day.head()

,electric_train.tm,electric_train.sum_qctr,electric_train.n,electric_train.sum_load,electric_train.n_mean_load,electric_train.elec
8761,2020-01-01 01:00:00,42250,58,4468.72,79.776051,96.58
35065,2020-01-01 01:00:00,16750,19,1802.20,97.138634,97.65
61369,2020-01-01 01:00:00,14600,11,2588.72,241.745562,97.35
105193,2020-01-01 01:00:00,16100,14,2691.52,186.873616,102.88
113977,2020-01-01 01:00:00,7750,10,1610.96,156.399551,103.00


In [55]:
local_split_idx = df_targets_2020_1am_day.index.to_list()
local_split_idx[:10]

[8761, 35065, 61369, 105193, 113977, 140281, 166585, 184129, 210433, 236737]

In [56]:
len(local_split_idx)

300

지역별로 나눌수 있음 